```kotlin
dependencies {

    implementation("androidx.navigation:navigation-compose:2.9.2")
    implementation("androidx.lifecycle:lifecycle-viewmodel-compose:2.9.2")
    ...
}
```

```kotlin
// =================================================================================
// --- Główna Aktywność i Nawigacja ---
// =================================================================================

class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        enableEdgeToEdge()
        setContent {
            MVVMStateFlowSharedFlowExampleTheme {
                Surface(modifier = Modifier.fillMaxSize()) {
                    val navController = rememberNavController()
                    NavHost(navController = navController, startDestination = "login") {
                        composable("login") {
                            LoginScreen(navController = navController)
                        }
                        composable("welcome") {
                            WelcomeScreen()
                        }
                    }
                }
            }
        }
    }
}

// =================================================================================
// --- Architektura (ViewModel, Stan, Zdarzenia) ---
// =================================================================================

// Definicja stanu UI dla ekranu logowania
data class LoginUiState(
    val email: String = "",
    val password: String = "",
    val isLoading: Boolean = false
)

// Definicja jednorazowych zdarzeń, które ViewModel może wysłać do UI
sealed class LoginEvent {
    object NavigateToWelcome : LoginEvent()
    data class ShowSnackbar(val message: String) : LoginEvent()
}

class LoginViewModel : ViewModel() {
    private val _uiState = MutableStateFlow(LoginUiState())
    val uiState: StateFlow<LoginUiState> = _uiState.asStateFlow()

    private val _eventFlow = MutableSharedFlow<LoginEvent>()
    val eventFlow: SharedFlow<LoginEvent> = _eventFlow.asSharedFlow()

    fun onEmailChange(email: String) {
        _uiState.update { it.copy(email = email) }
    }

    fun onPasswordChange(password: String) {
        _uiState.update { it.copy(password = password) }
    }

    fun onLoginClicked() {
        viewModelScope.launch {
            _uiState.update { it.copy(isLoading = true) }
            delay(2000) // Symulacja zapytania sieciowego

            if (uiState.value.email == "test@test.com" && uiState.value.password == "1234") {
                // Sukces: wyślij zdarzenie nawigacji
                _eventFlow.emit(LoginEvent.NavigateToWelcome)
            } else {
                // Błąd: wyślij zdarzenie Snackbar
                _eventFlow.emit(LoginEvent.ShowSnackbar("Nieprawidłowe dane logowania."))
            }
            _uiState.update { it.copy(isLoading = false) }
        }
    }
}

// =================================================================================
// --- Ekrany ---
// =================================================================================

@Composable
fun LoginScreen(
    navController: NavController,
    viewModel: LoginViewModel = viewModel()
) {
    val uiState by viewModel.uiState.collectAsStateWithLifecycle()
    val snackbarHostState = remember { SnackbarHostState() }

    // Ten LaunchedEffect nasłuchuje na jednorazowe zdarzenia z ViewModelu
    LaunchedEffect(Unit) {
        viewModel.eventFlow.collect { event ->
            when (event) {
                is LoginEvent.NavigateToWelcome -> {
                    // Usuń ekran logowania z back stacka, aby nie można było do niego wrócić
                    navController.navigate("welcome") {
                        popUpTo("login") { inclusive = true }
                    }
                }
                is LoginEvent.ShowSnackbar -> {
                    snackbarHostState.showSnackbar(
                        message = event.message,
                        duration = SnackbarDuration.Short
                    )
                }
            }
        }
    }

    Scaffold(snackbarHost = { SnackbarHost(snackbarHostState) }) { padding ->
        Box(
            modifier = Modifier.fillMaxSize().padding(padding),
            contentAlignment = Alignment.Center
        ) {
            Column(
                modifier = Modifier.padding(32.dp),
                horizontalAlignment = Alignment.CenterHorizontally
            ) {
                Text("Zaloguj się", style = MaterialTheme.typography.headlineLarge)
                Spacer(Modifier.height(24.dp))
                OutlinedTextField(
                    value = uiState.email,
                    onValueChange = viewModel::onEmailChange,
                    label = { Text("Email") },
                    modifier = Modifier.fillMaxWidth()
                )
                Spacer(Modifier.height(16.dp))
                OutlinedTextField(
                    value = uiState.password,
                    onValueChange = viewModel::onPasswordChange,
                    label = { Text("Hasło") },
                    modifier = Modifier.fillMaxWidth()
                )
                Spacer(Modifier.height(24.dp))
                Button(
                    onClick = viewModel::onLoginClicked,
                    enabled = !uiState.isLoading,
                    modifier = Modifier.fillMaxWidth()
                ) {
                    Text("Zaloguj")
                }
                if (uiState.isLoading) {
                    Spacer(Modifier.height(16.dp))
                    CircularProgressIndicator()
                }
            }
        }
    }
}

@Composable
fun WelcomeScreen() {
    Box(
        modifier = Modifier.fillMaxSize(),
        contentAlignment = Alignment.Center
    ) {
        Text("Witaj!", style = MaterialTheme.typography.displayMedium)
    }
}
```